## 梳理gpt-researcher的代码架构

### 根目录上有以下几个文件夹
- [actions](https://github.com/assafelovic/gpt-researcher/tree/master/actions)
    - [web_scrape.py](https://github.com/assafelovic/gpt-researcher/blob/master/actions/web_scrape.py)
        -  [async_browse(url, question, websocket)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/actions/web_scrape.py#L36C24-L36C69): 返回`当前url网页`内`感知question`的`summary_text`
            - 8线程池, `3个任务`加入线程池
                - [scrape_text_with_selenium(url)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/actions/web_scrape.py#L99): 该函数也在`web_scrape.py`内
                - add_header(driver)
                - summary.summarize_text(url, text, question, driver): `TODO`: 原始的summary prompt只有user没有system特别简单，总结效果很差，需要修改
        - [browse_website(url, question)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/actions/web_scrape.py#L69): 是上面async_browse(url, question, websocker)的`非异步版本`
            - scrape_text_with_selenium(url)
            - add_header(driver)
            - summary.summarize_text(url, text, question, driver)
            - links = scrape_links_with_selenium(driver, url)
            - close_browser(driver)
        - [scrape_text_with_selenium(url)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/actions/web_scrape.py#L99): 对url进行抓取获取页面text
            - 根据.env内设置的浏览器(firefox)得到driver
            - 浏览器在headless模式下获取到url页面
            - [get_text(soup)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/actions/web_scrape.py#L163): 提取页面的整体文本text
            - 返回driver和text
        - [scrape_links_with_selenium(driver, url)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/actions/web_scrape.py#L179C5-L179C31): 获取url内所有的链接
        - [close_browser(driver)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/actions/web_scrape.py#L199C5-L199C37): 用来关闭浏览器
        - add_header(driver): 运行一个js脚本: driver.execute_script("overlap.js"). `TODO`: 这个overlap.js的功能是什么?
    - [web_search.py](https://github.com/assafelovic/gpt-researcher/blob/master/actions/web_search.py): py文件内部只有一个函数`web_search(query, num_results=4)`
        - `results = ddgs.text(query)`: 给`duckduckgo(DDGS)`搜索引擎输入query，拿到搜索结果`search_results`, 这个搜索引擎`需要翻墙`
- [agent](https://github.com/assafelovic/gpt-researcher/tree/master/agent)
    - [llm_utils.py](https://github.com/assafelovic/gpt-researcher/blob/master/agent/llm_utils.py)
        - [create_chat_completion(messages, model, temperature, max_tokens, stream, websocket)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/llm_utils.py#L23)
            - [send_chat_completion_request(messages, model, temperature, max_tokens, stream, websocket)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/llm_utils.py#L61)
                - [stream_response(model, messages, temperature, max_tokens, websocket)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/llm_utils.py#L77C11-L77C26): 只有在stream=True时会调用该函数
                - stream=False时, 直接调用`lc_openai.ChatCompletion.create()`
        - [choose_agent(task)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/llm_utils.py#L101)
            - [create_chat_completion]()
    - [prompts.py](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py)
        - [generate_agent_role_prompt(agent)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L2)
            - 使用1: [permchain_example/writer_actors/writer.py](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/permchain_example/writer_actors/writer.py#L13)
            - 使用2: [ResearchAgent初始化时self.agent_role_prompt](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/research_agent.py#L34)
        - [generate_report_prompt(question, research_summary)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L19C5-L19C56): `生成research_report`
            - 使用1: [permchain_example/writer_actors/writer.py](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/permchain_example/writer_actors/writer.py#L14C27-L14C27)
            - 使用2: 在下面的`ResearchAgent.write_report(report_type, websocket=None)内report_type_mapping`内使用
        - [generate_search_queries_prompt(question)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L34C5-L34C45)
            - 功能: 输入question，让GPT4提出几个query问题
            - 使用1: permchain
            - 使用2: [research_agent.create_search_queries()](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/research_agent.py#L98)函数内使用GPT4来产生多个queries
        - [generate_resource_report_prompt(question, research_summary)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L45C5-L45C64): 生成resource_report
            - `生成resource_report`: 在下面的`ResearchAgent.write_report(report_type, websocket=None)内report_type_mapping`内使用
        - [generate_outline_report_prompt(question, research_summary)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L64C5-L64C63): 
            - `生成outline_report`: 在下面的`ResearchAgent.write_report(report_type, websocket=None)内report_type_mapping`内使用
        - [generate_concepts_prompt(question, research_summary)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L77C5-L77C57):
            - 使用1: 在[ResearchAgent.create_concepts()](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/research_agent.py#L153)中有调用: `result = self.call_agent(prompts.generate_concepts_prompt(self.question, self.research_summary))`
        - [generate_lesson_prompt(concept)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L89C5-L89C37):
            - 使用1: 在[ResearchAgent.write_lessons()](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/research_agent.py#L180)内有调用: `answer = await self.call_agent(prompts.generate_lesson_prompt(concept), stream=True)`
        - [get_report_by_type(report_type)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L104C5-L104C36)
            - 使用1: 在[ResearchAgent.write_report(report_type, websocket=None)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/research_agent.py#L168C9-L168C67)调用: `report_type_func = prompts.get_report_by_type(report_type)`
            - ```
                report_type_mapping = {
                    'research_report': generate_report_prompt: question, research_summary
                    'resource_report': generate_resource_report_prompt: question, research_summary
                    'outline_report': generate_outline_report_prompt: question, research_summary
                }
                ```     
            - [generate_report_prompt](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L19): question, research_summary
            - [generate_resource_report_prompt](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L45)
            - [generate_outline_report_prompt](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L64)
            
        - [auto_agent_instructions()](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L112)
            - 这个prompt没有接收任何输入
            - 仅在[permchain_example/search_actors/gpt_researcher.py](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/permchain_example/search_actors/gpt_researcher.py#L23)内使用
    - [research_agent.py](https://github.com/assafelovic/gpt-researcher/blob/master/agent/research_agent.py)
        - [ResearchAgent类](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/research_agent.py#L25)
            - 初始化时需要传入: ResearchAgent(question, agent, agent_role_prompt, websocket=None)
                - question: 当前针对哪个question进行科研，则为这个科研问题创建一个ResearchAgent
                - agent: 实际是agent_name, 由GPT4根据当前question生成
                - agent_role_prompt: 当前agent的角色prompt, 同样由GPT4生成
            - async def `summarize(text, topic)`: 调用[processing/text.py内的create_message(chunk, question)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/processing/text.py#L117)
                - 这个message是用于给定contex中找答案的，找不到答案则summary，过于简单，很容易得到"xx没有提及"这类回复
                - 这个message内只有一个role="user"，缺少role="system"
                - ```
                    return {
                        "role": "user",
                        "content": f'"""{chunk}""" Using the above text, answer in short the following'
                        f' question: "{question}" -- if the question cannot be answered using the text,'
                        " simply summarize the text. "
                        "Include all factual information, numbers, stats etc if available.",
                    }
                ```
            - async def `call_agent(action, stream=False, websocket=None)`: 
                - messages构建时:
                    - system=self.agent_role_prompt
                    - user=输入参数action
                - 调用create_chat_completion(model, messages, stream, websocket)得到回答GPT4的answer
                - 返回answer
            - async def `create_search_queries()`: 根据用户输入task，GPT4自行生成几个search_queries
                - 构造该任务的prompt: `prompts.generate_search_queries_prompt(self.question)`, 作为user_prompt
                - 调用GPT4得到结果: `result = await self.call_agent(prompts.generate_search_queries_prompt(self.question))`
                    - system_prompt是当前ResearchAgent初始化时就设置好的
                    - user_prompt是`prompts.generate_search_queries_prompt(self.question)`得到的
            - async def `stream_output(output)`: 
                - 如果没有self.websocket，则直接print(output)
                - 有self.websocket则`self.websocket.send_json({"type": "logs", "output": output})`
            - async def `async_search(query)`:
                - 调用[web_search(query, num_results=4)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/actions/web_search.py#L7)得到query在duckduckgo搜索引擎中找到的网页结果, 该函数返回的是json字符串(解开是个list)
                - 解开刚才搜索引擎得到的结果列表: `search_results = json.loads(web_search(query))`, 为list
                - 拿到所有未浏览过的urls, 称为`new_search_urls`
                - stream_output输出这些链接到前端
                - 所有这些urls调用`异步async_browse`(url, query, self.websocket): 
                    - 用`asyncio.gather()`(每个浏览页面是相互独立的，完全适合异步)搜集到所有任务的浏览结果
                    - 每一个async_browse(url, query)返回一个summary_text
                    - ```
                        tasks = [async_browse(url, query, self.websocket) for url in await new_search_urls]
                        responses = await asyncio.gather(*tasks, return_exceptions=True)
                        ```
                    - 返回responses: list of summary_text
            - async def `run_search_summary(query)`: 一个query会涉及多个url网页的浏览后总结，concat所有网页的summary_text
                - 先打印log, 当前基于哪个query在做search_summary: self.stream_output(f"🔎 Running research for '{query}'...")
                - responses = await self.`async_search(query)`: 异步浏览该query得到的所有urls，浏览后返回它们的summary_text
                    - 内部[调用了async_browse](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/research_agent.py#L113C28-L113C28)(url, question): 这个函数内部会做[summarize_text(url, text(页面全部文本), question)](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/actions/web_scrape.py#L57)
                - `result = "\n".join(responses)`: 将所有urls页面的summary_text连接起来
                - 写入硬盘文件，并且返回result, 也就是这个search_query的搜索结果
            - async def `conduct_research()`:
                - 先检查当前question的dir_path下是否已经有这个科研问题的多个search_queries的`research-{search_query}.txt`文件，存在则直接加载成`self.research_summary`
                - self.research_summary为空时, 
                    - 先让GPT4对当前科研问题生成一些search_queries = await self.create_search_queries()
                    - 对每个search_query调用`research_result = await self.run_search_summary(query)`得到这个search_query的多个urls总结summary_text
                    - concat所有search_query的summary_text作为self.research_summary
                    - 返回self.research_summary
                - 这个ResearchAgent.conduct_research()在[agent/run.py内被调用](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/run.py#L50), 调用后此时assistant.research_summary就有值了，而且是多个search_queries的summary_text合起来
                - 再调用`assistant.write_report(report_type, websocket)`，收集到了科研资料，就可以输出特定report_type的报告
            - async def `write_report(report_type, websocket=None)`: 
                - 根据report_type，调用不同的prompt_func, 输入: 当前科研问题question, 和收集好的self.research_summary，就可以通过[`3种写报告prompt`](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/agent/prompts.py#L19)，让GPT4返回写好的报告
                - ```
                    report_type_func = prompts.get_report_by_type(report_type)
                    answer = await self.call_agent(report_type_func(self.question, self.research_summary),
                                                stream=websocket is not None, websocket=websocket)
                    ```
                - 注意，这里的call_agent(action)传入的action=user_prompt，system_prompt是ResearchAgent.agent_role_prompt在初始化时就确定了
    - [run.py](https://github.com/assafelovic/gpt-researcher/blob/master/agent/run.py)
        - WebSocketManager类
- client: web端
- [config](https://github.com/assafelovic/gpt-researcher/tree/master/config): 里面实现了Config类, 它是单例
- js: web端
    - [overlay.js](https://github.com/assafelovic/gpt-researcher/blob/38fd33123492420d939139092ae4baac34075c21/js/overlay.js)
- [permchain_example](https://github.com/assafelovic/gpt-researcher/tree/master/permchain_example): Sample use of `Langchain's Autonomous agent framework` [Permchain](https://github.com/langchain-ai/permchain) with GPT Researcher.
- [processing](https://github.com/assafelovic/gpt-researcher/tree/master/processing)
    - [html.py](https://github.com/assafelovic/gpt-researcher/blob/master/processing/html.py)
    - [text.py](https://github.com/assafelovic/gpt-researcher/blob/master/processing/text.py)
